In [1]:
from keras.models import load_model
import glob
import cv2
import pandas as pd
import pickle

c:\users\yq\anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
test_path = glob.glob("test/*")

mean = pickle.load(open("mean.h5", "rb"))
le = pickle.load(open("le.h5", "rb"))
model1 = load_model("epoch170_resnet4.h5")
model2 = load_model("epoch170_resnet5.h5")

c:\users\yq\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.19.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
import numpy as np
X = []
for i in test_path:
    img = cv2.imread(i)
    img = cv2.resize(img, (224, 224))
    X.append(img)
    
X = np.array(X).astype(np.float32)
X -= mean

X.shape

(794, 224, 224, 3)

In [6]:
pred1 = model1.predict(X)
pred2 = model2.predict(X)

pred = pred1 + pred2
pred.shape

(794, 12)

In [7]:
pred = np.argmax(pred, axis=-1)

In [8]:
df = pd.DataFrame({"file": [xx.split("\\")[-1] for xx in test_path],
                  "species": le.inverse_transform(pred)})
df.head()

c:\users\yq\anaconda3\envs\tf\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,file,species
0,0021e90e4.png,Small-flowered Cranesbill
1,003d61042.png,Fat Hen
2,007b3da8b.png,Sugar beet
3,0086a6340.png,Common Chickweed
4,00c47e980.png,Sugar beet


In [9]:
df.to_csv("ensemble.csv", index=False)